In [1]:
%clear
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.integrate import odeint
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.optimize import least_squares

%matplotlib inline

mpl.rcParams['figure.dpi']=100
mpl.rcParams['figure.titlesize']=20
mpl.rcParams['axes.facecolor']='white'        
mpl.rcParams['lines.linewidth']=2.0
mpl.rcParams['axes.linewidth']=2.0
mpl.rcParams['xtick.major.pad']=6
mpl.rcParams['ytick.major.pad']=6
mpl.rcParams['xtick.labelsize']=14
mpl.rcParams['ytick.labelsize']=14
mpl.rcParams['axes.titlesize']=18
mpl.rcParams['axes.labelsize']=18
mpl.rcParams['axes.grid']='True'
mpl.rcParams['axes.axisbelow']='line'
mpl.rcParams['legend.fontsize']=12

In [2]:
import pandas as pd

folder="~/PHD/NOTEBOOK/Data_exp_Kv/"

# Activation dataset

dataset_act = pd.read_csv(folder+"Act_datasets.csv", skiprows=1)

act_data = dataset_act.to_numpy()

act_data_WT = act_data[:,1]

act_data_S390N = act_data[:,3]

# Inactivation dataset

dataset_inact = pd.read_csv(folder+"Inact_datasets.csv", skiprows=1)

inact_data = dataset_inact.to_numpy()

inact_data_WT = inact_data[:,1]

inact_data_S390N = inact_data[:,3]

# Recovery dataset

dataset_rec = pd.read_csv(folder+"Rec_datasets.csv", skiprows=1)

rec_data = dataset_rec.to_numpy()

rec_data_WT = rec_data[:,1]

rec_data_S390N = rec_data[:,3]


In [3]:
def ode_rec_mut (C, t, p):
    
    C0=C[0]
    C1=C[1]
    C2=C[2]
    C3=C[3]
    C4=C[4]
    I0=C[5]
    I1=C[6]
    I2=C[7]
    I3=C[8]
    I4=C[9]
    O=C[10]

    
    #constants
    
    T = 291.0 #K or 18 degree celsius
    e =  1.602176634 * (10**-19.0) # C
    K_B = 1.380649 * (10**-23.0) # J*K^-1
    
    exp_factor = (e/(K_B * T)) * (10**-3) 

    #Voltage sequences
    
    V = 0.0 #mV 

    if 0 <= t < 1.00:
        V=-90.0
        
    if 1.00 <= t < 2.00:
        V=60.0
        
    if 2.00 <= t <= (2.00+p[6]):
        V=-90.0
        
    if (2.00+p[6]) < t <= (p[6]+3.00):
        V=60.0
 
    if t > (p[6]+3.00):
        V=-90.0

    # S390N parameters

    alpha_0 = 3.35079341e+03 #s^-1
#    alpha_1 = 3.28685357e-01   #s^-1
    
    beta_0 = 1.05273741e+02 #s^-1
#    beta_1 = 1.73456113e+00  #s^-1
    
    k_CO_0 = 6.76097035e+02 #s^-1
#    k_CO_1 = 1.02986592e-01  #s^-1
    
    k_OC_0 = 8.55022238e+01 #s^-1
    k_OC_1 = 1.18068777e-03   #s^-1
    
#    k_CI = 1.80092357e+02 #s^-1
#    k_IC = 3.89986657e-01   #s^-1 0.20
    
#    f = 4.31533471e-01 # 0.31 

    k_CI = p[3]
    
    k_IC = p[4]

    f = p[5]
    
    #voltage dependent rate constants
    
    #alpha = alpha_0 * np.exp(alpha_1 * (V * exp_factor))
    #beta = beta_0 * np.exp(-1.0 * beta_1 * (V * exp_factor))
    #k_CO = k_CO_0 * np.exp(k_CO_1 * (V * exp_factor))
    #k_OC = k_OC_0 * np.exp(-1.0 * k_OC_1 * (V * exp_factor))
    
     
    alpha = alpha_0 * np.exp(p[0] * (V * exp_factor))
    beta = beta_0 * np.exp(-1.0 * p[1] * (V * exp_factor))
    k_CO = k_CO_0 * np.exp(p[2] * (V * exp_factor))
    k_OC = k_OC_0 * np.exp(-1.0 * k_OC_1 * (V * exp_factor))
    
    
    # ODEs
    
    dC0dt = beta * C1 + (k_IC/(f**4.0)) * I0 - (k_CI*(f**4.0) + 4.0 * alpha) * C0
    dC1dt = 4.0 * alpha * C0 + 2.0 * beta * C2 + (k_IC/(f**3.0)) * I1 - (k_CI*(f**3.0) + beta + 3.0 * alpha) * C1 
    dC2dt = 3.0 * alpha * C1 + 3.0 * beta * C3 + (k_IC/(f**2.0)) * I2 - (k_CI*(f**2.0) + 2.0 * beta + 2.0 * alpha) * C2 
    dC3dt = 2.0 * alpha * C2 + 4.0 * beta * C4 + (k_IC/f) * I3 - (k_CI*f + 3.0 * beta + 1.0 * alpha) * C3 
    dC4dt = 1.0 * alpha * C3 + k_OC * O + k_IC * I4 - (k_CI + k_CO + 4.0 * beta) * C4 
    
    dI0dt = beta * f * I1 + (k_CI*(f**4.0)) * C0 - (k_IC/(f**4.0) + 4.0 * (alpha/f)) * I0
    dI1dt = 4.0 * (alpha/f) * I0 + 2.0 * beta * f * I2 + (k_CI*(f**3.0)) * C1 - (k_IC/(f**3.0) + beta * f + 3.0 * (alpha/f)) * I1 
    dI2dt = 3.0 * (alpha/f) * I1 + 3.0 * beta * f * I3 + (k_CI*(f**2.0)) * C2 - (k_IC/(f**2.0) + 2.0 * beta * f + 2.0 * (alpha/f)) * I2 
    dI3dt = 2.0 * (alpha/f) * I2 + 4.0 * beta * f * I4 + (k_CI*f) * C3 - (k_IC/f + 3.0 * beta * f + 1.0 * (alpha/f)) * I3 
    dI4dt = 1.0 * (alpha/f) * I3 + k_CI * C4 - (k_IC + 4.0 * beta * f) * I4     
    
    dOdt = k_CO * C4 - k_OC * O
    
    return (dC0dt, dC1dt, dC2dt, dC3dt, dC4dt, dI0dt, dI1dt, dI2dt, dI3dt, dI4dt, dOdt)

In [4]:
def Rec_Protocol(max_t, Deltat):
    min_t = 0.015 #mV
    
    t_pulse = np.linspace(min_t,max_t,np.abs(np.int((max_t-min_t)/Deltat))+1)
    
    return t_pulse #array of testing prepulses

In [5]:
def Simple_exp_eq(x,tau):
    return 1.0 - np.exp(-x / tau)

In [6]:
def Rec_LS_func(x, teta):

    # conductance parameters

    EK      = -90.0    # mV
    gK_max  = 15.9     # nS 

    # Assuming no leaking
    #EL      = 0.0    # mV
    #gL_max  = 0.0    # mS 

    # Membrane capacitance
    Cm      = 1.0    # microF cm^-2
    
    #define activation sequence protocol
    
    tmax = 0.300 #s
    
    increment = 0.015 #s
    
    t_pulse = Rec_Protocol(tmax, increment)
    
    # Time discretiztion
    tini_prep = 1.00 #s

    tend_prep = 2.00 #s

    Vtesting = 60.0 #mV

    # Time discretiztion

    tend = 4.00 #s
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    
    # time array
    #t = np.linspace(tini,tend,Npoints)

    # prepare empty arrays
    Open_states = np.zeros((Npoints,len(t_pulse)))

    max_conductance = np.zeros(len(t_pulse)) 

    max_currents = np.zeros(len(t_pulse))

    max_conductance_prep = np.zeros(len(t_pulse))

    max_currents_prep = np.zeros(len(t_pulse))

    
    for i in range (0,len(t_pulse)):

        gamma = np.append(teta, t_pulse[i])
        
        f = lambda S,t: ode_rec_mut(S, t, gamma)
        
        r = odeint(f, S0, x)
        
        Open_states[:,i] = r[:,10]
        
        max_conductance[i] = gK_max * np.amax(r[np.int(Points_per_sec*(tend_prep+t_pulse[i])):,10])

        max_conductance_prep[i] = gK_max * np.amax(r[np.int(Points_per_sec*tini_prep):np.int(Points_per_sec*tend_prep),10])

        # Compute the current proportional to the open channel conductance and potential applied

        max_currents[i] = max_conductance[i] * (Vtesting - EK)

        max_currents_prep[i] = max_conductance_prep[i] * (Vtesting - EK) # nS * mV = pA

    I_Imax = np.true_divide(max_currents,max_currents_prep)

    return (I_Imax)
  

In [7]:
def residual_Rec(p):
    
    #define activation sequence protocol
    
    tmax = 300.0 #ms
    
    increment = 15.0 #ms
    
    t_pulse = Rec_Protocol(tmax, increment)
    
    #experimental data for hemiactivation and slope
    
#    exp_tau = 47.0 #ms
    
    
    #evaluation of experimental points trough the declared Boltzmann fit
    
#    exp_data = Simple_exp_eq(t_pulse,exp_tau)
    
    # Time discretiztion
    tini = 0.0
    
    tend = 4.00
    
#    ttest = 2.00 #time at which you start record the current
     
    Npoints = 200000
    
    Points_per_sec = np.int(Npoints/tend) 
    
    # time array
    t = np.linspace(tini,tend,Npoints)
    
    sq_err = np.sum(np.subtract(rec_data_S390N,Rec_LS_func(t,p))**2)
    
    Rec_cost_func = (1.0/len(t_pulse)) * sq_err
    
    return Rec_cost_func

## Debugger Recovery

In [11]:
# Main 
import datetime
from scipy.optimize import minimize

# Guess of initial conditions for the states 
C0_0=0.4390
C1_0=0.2588
C2_0=0.0572
C3_0=0.0056
C4_0=0.0002
I0_0=0.0128
I1_0=0.0553
I2_0=0.0894
I3_0=0.0642
I4_0=0.0172
O_0=0.0001

# Pack up the initial conditions:

S0 = [C0_0, C1_0, C2_0, C3_0, C4_0, I0_0, I1_0, I2_0, I3_0, I4_0, O_0]

# guess of the transition rates and other ODEs params (start from Lin Model)

#    alpha_1 = 3.28685357e-01   #s^-1

#    beta_1 = 1.73456113e+00  #s^-1
    
#    k_CO_1 = 1.02986592e-01  #s^-1
        
#    k_CI = 1.80092357e+02 #s^-1
#    k_IC = 3.89986657e-01   #s^-1 0.20
    
#    f = 4.31533471e-01 # 0.31 
guess = np.array([3.28685357e-01, 1.73456113e+00, 1.02986592e-01, 1.80092357e+02, 3.89986657e-01, 4.31533471e-01])

# set boundaries for parameters lsq fitting based on physical reasoning

#boundaries = ((400.0,1000.0),(0.0,1.0),(0.0,100.0),(0.0,10.0),(0.0,1000.0),(0.0,1.0),(0.0,100.0),(0.0,1.0),(0.0,100.0),(0.0,1.0),(0,0.5))

# Run the optimization for the cost function 

#result_lm = least_squares(residual_Act, guess, method='lm')

#result = minimize(residual_Act, guess, bounds=boundaries, method='L-BFGS-B')
print(datetime.datetime.now())
result = minimize(residual_Rec, guess, method='Nelder-Mead', options={'maxiter':1000,'maxfev':10000})
print(datetime.datetime.now())
print (result)

2020-06-14 02:36:29.471665


KeyboardInterrupt: 